In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(dplyr)
library(randomForest)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin



In [2]:
set.seed(1)

In [3]:
jRoadType <- c("都市間高速", "都市高速", "有料道路", "国道", "県道", "主要地方道", "一般道1、一般道2、一般道3", "その他")
jHighSpeeds <- c("都市間高速", "都市高速", "有料道路")

In [4]:
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")

In [5]:
printf <- function(...) cat(sprintf(...))

In [6]:
CV <- function(dfx, verbose=FALSE) {        
    folds <- createFolds(dfx$flag, k=2)
    
    count <- 1
    errs <- c()
    reds <- c()
    blues <- c()
    shorts <- c()
    for (ids in folds) {
        train <- dfx[-ids, c(valuables, "flag")]
        test <- dfx[ids, ]
        fit <- randomForest(flag ~ ., data=train, mtry=3, ntree=300) 
        p <- predict(fit, newdata=test)
        
        if (verbose) {
            printf("Fold%d\n", count)
            result <- correctVsPredict(test, p, verbose)
            shorts <- c(shorts, shortErr(test, p))
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
            count <- count + 1
            printf("\n")
        } else {
            result <- correctVsPredict(test, p, verbose)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
        }
        
        nerr <- sum(p != test$flag)
        errs <- c(errs, nerr / nrow(test))
    }
    
    return(c(mean(errs), mean(shorts), mean(reds), mean(blues)))
}

In [7]:
shortErr <- function(test, predictedFlags) {
    predictedRedRows <- test[predictedFlags == "Red", ]
    nRedAccel <- nrow(filter(predictedRedRows, flag == "Red" & AccelOrBrake == "Accel"))
    nBlueAccel <- nrow(filter(predictedRedRows, flag == "Blue" & AccelOrBrake == "Accel"))
    nPredictedRedsAB <- nRedAccel + nBlueAccel
    printf("Short: accel/predictRedAB = %d/%d = %f\n", nRedAccel, nPredictedRedsAB, nRedAccel / nPredictedRedsAB)
    # print(predictedRedRows %>% group_by(flag, AccelOrBrake) %>% summarize(count=n()))
    nRedAccel / nPredictedRedsAB
}

In [8]:
correctVsPredict <- function(test, predictedFlags, verbose=FALSE) {    
    # for All
    # nTests <- nrow(test)
    # nCorrectAll <- sum((predictedFlags == test$flag) == TRUE)
    # printf("As fo All: correct/all = %d/%d = %f\n", nCorrectAll, nTests, nCorrectAll / nTests)
    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum((predictedRedRows$flag == 'Red') == TRUE)
    nPredictedReds <- nrow(predictedRedRows)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum((predictedBlueRows$flag == 'Blue') == TRUE)
    nPredictedBlues <- nrow(predictedBlueRows)
        
    if (verbose) {
        printf("As for Red: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds / nPredictedReds)
        printf("As for Blue: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues / nPredictedBlues)  
    }    
    
    c(nCorrectReds/nPredictedReds, nCorrectBlues/nPredictedBlues)
}

In [9]:
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

In [10]:
createFormula <- function(invalids, allFeatures) {
    isValidColumn <- function(x) ! x %in% invalids
    
    features <- Filter(isValidColumn, allFeatures)
    return(paste("flag ~ ", paste(features, collapse=" + ")))
}

# Predict Reds

In [11]:
df3 <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [12]:
df3$AccelOrBrake[df3$flag == "RedA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "RedB"] <- "Brake"
df3$AccelOrBrake[df3$flag == "BlueA"] <- "Accel"
df3$AccelOrBrake[df3$flag == "BlueB"] <- "Brake"

In [13]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [14]:
allFeatures <- c(colnames(df3))

In [15]:
roadTypes <- unique(df3$RoadType)

In [16]:
df <- df3[, c(valuables, "AccelOrBrake", "flag")]

In [17]:
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df[df$RoadType == i, ]
    result <- CV(dfx, verbose=TRUE)
    printRedRatios(dfx)
    printf("Red: Mean correct/predict = %f\n", result[3])
    printf("Blue: Mean correct/predict = %f\n", result[4])
    printf("Short: Mean accel/predictRed = %f\n", result[2]) 
    printf("CV value: %f", result[1])
    printf("\n\n")
}

RoadType: 7 (その他)
Fold1
As for Red: correct/predict = 12/13 = 0.923077
As for Blue: correct/predict = 0/0 = NaN
Short: accel/predictRedAB = 3/3 = 1.000000

Fold2
As for Red: correct/predict = 13/14 = 0.928571
As for Blue: correct/predict = 0/0 = NaN
Short: accel/predictRedAB = 2/2 = 1.000000

Red/All = 25/27 = 0.925926
1 - Red/All = 2/27 = 0.074074
Red: Mean correct/predict = 0.925824
Blue: Mean correct/predict = NaN
Short: Mean accel/predictRed = 1.000000
CV value: 0.074176

RoadType: 6 (一般道1、一般道2、一般道3)
Fold1
As for Red: correct/predict = 22/43 = 0.511628
As for Blue: correct/predict = 28/51 = 0.549020
Short: accel/predictRedAB = 10/10 = 1.000000

Fold2
As for Red: correct/predict = 26/35 = 0.742857
As for Blue: correct/predict = 41/60 = 0.683333
Short: accel/predictRedAB = 13/13 = 1.000000

Red/All = 90/189 = 0.476190
1 - Red/All = 99/189 = 0.523810
Red: Mean correct/predict = 0.627243
Blue: Mean correct/predict = 0.616176
Short: Mean accel/predictRed = 1.000000
CV value: 0.381411

R

# Divide More ...

In [18]:
df %>% group_by(RoadType) %>% summarize(counts=n())

RoadType,counts
0,81
2,47
3,324
4,71
5,303
6,189
7,27


# Threshold = 10, 30

In [19]:
set.seed(1)

In [20]:
# Not Use
dfx <- df[df$RoadType == i, ]
folds <- createFolds(dfx$flag, k=5)

In [21]:
verbose = TRUE

In [22]:
i = 6
dfx <- df[df$RoadType == i, ]
folds <- createFolds(dfx$flag, k=5)

In [23]:
i = 4
dfx <- df[df$RoadType == i, ]
folds <- createFolds(dfx$flag, k=5)

In [24]:
i = 5
threshold1 <- 10
threshold2 <- 30
printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
dfx <- df[df$RoadType == i, ]
dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
dfx3 <- dfx %>% filter(AverageVelocity > threshold2)

result1 <- CV(dfx1, verbose=verbose)        
printf("AverageVelocity <= %d:\n", threshold1)
printRedRatios(dfx1)        
printf("Red: Mean correct/predict = %f\n", result[3])
printf("Blue: Mean correct/predict = %f\n", result[4])
printf("Short: Mean accel/predictRed = %f\n", result[2]) 
printf("CV value: %f", result1[1])        
printf("\n\n")
        
result2 <- CV(dfx2, verbose=verbose)
printf("%d < AverageVelocity <= %d:\n", threshold1, threshold2)
printRedRatios(dfx2)
printf("Red: Mean correct/predict = %f\n", result[3])
printf("Blue: Mean correct/predict = %f\n", result[4])
printf("Short: Mean accel/predictRed = %f\n", result[2]) 
printf("CV value: %f", result2[1])       
printf("\n\n")

result3 <- CV(dfx3, verbose=verbose)        
printf("AverageVelocity > %d:\n", threshold2)
printRedRatios(dfx3)
printf("Red: Mean correct/predict = %f\n", result[3])
printf("Blue: Mean correct/predict = %f\n", result[4])
printf("Short: Mean accel/predictRed = %f\n", result[2]) 
printf("CV value: %f", result2[1])       
printf("\n\n")

RoadType: 5 (主要地方道)
Fold1
As for Red: correct/predict = 8/15 = 0.533333
As for Blue: correct/predict = 11/20 = 0.550000
Short: accel/predictRedAB = 3/3 = 1.000000

Fold2
As for Red: correct/predict = 8/19 = 0.421053
As for Blue: correct/predict = 7/15 = 0.466667
Short: accel/predictRedAB = 4/4 = 1.000000

AverageVelocity <= 10:
Red/All = 33/69 = 0.478261
1 - Red/All = 36/69 = 0.521739
Red: Mean correct/predict = 0.628289
Blue: Mean correct/predict = 0.242857
Short: Mean accel/predictRed = 0.900000
CV value: 0.507983

Fold1
As for Red: correct/predict = 28/45 = 0.622222
As for Blue: correct/predict = 28/45 = 0.622222
Short: accel/predictRedAB = 16/19 = 0.842105

Fold2
As for Red: correct/predict = 29/46 = 0.630435
As for Blue: correct/predict = 27/42 = 0.642857
Short: accel/predictRedAB = 18/20 = 0.900000

10 < AverageVelocity <= 30:
Red/All = 89/178 = 0.500000
1 - Red/All = 89/178 = 0.500000
Red: Mean correct/predict = 0.628289
Blue: Mean correct/predict = 0.242857
Short: Mean accel/pr

In [25]:
i = 3
threshold1 <- 10
threshold2 <- 30
printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
dfx <- df[df$RoadType == i, ]
dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
dfx3 <- dfx %>% filter(AverageVelocity > threshold2)

result1 <- CV(dfx1, verbose=verbose)        
printf("AverageVelocity <= %d:\n", threshold1)
printRedRatios(dfx1)        
printf("Red: Mean correct/predict = %f\n", result[3])
printf("Blue: Mean correct/predict = %f\n", result[4])
printf("Short: Mean accel/predictRed = %f\n", result[2]) 
printf("CV value: %f", result1[1])        
printf("\n\n")
        
result2 <- CV(dfx2, verbose=verbose)
printf("%d < AverageVelocity <= %d:\n", threshold1, threshold2)
printRedRatios(dfx2)
printf("Red: Mean correct/predict = %f\n", result[3])
printf("Blue: Mean correct/predict = %f\n", result[4])
printf("Short: Mean accel/predictRed = %f\n", result[2]) 
printf("CV value: %f", result2[1])       
printf("\n\n")

result3 <- CV(dfx3, verbose=verbose)        
printf("AverageVelocity > %d:\n", threshold2)
printRedRatios(dfx3)
printf("Red: Mean correct/predict = %f\n", result[3])
printf("Blue: Mean correct/predict = %f\n", result[4])
printf("Short: Mean accel/predictRed = %f\n", result[2]) 
printf("CV value: %f", result2[1])       
printf("\n\n")

RoadType: 3 (国道)
Fold1
As for Red: correct/predict = 13/21 = 0.619048
As for Blue: correct/predict = 1/5 = 0.200000
Short: accel/predictRedAB = 12/13 = 0.923077

Fold2
As for Red: correct/predict = 14/20 = 0.700000
As for Blue: correct/predict = 3/6 = 0.500000
Short: accel/predictRedAB = 13/15 = 0.866667

AverageVelocity <= 10:
Red/All = 34/52 = 0.653846
1 - Red/All = 18/52 = 0.346154
Red: Mean correct/predict = 0.628289
Blue: Mean correct/predict = 0.242857
Short: Mean accel/predictRed = 0.900000
CV value: 0.403846

Fold1
As for Red: correct/predict = 41/60 = 0.683333
As for Blue: correct/predict = 10/20 = 0.500000
Short: accel/predictRedAB = 33/37 = 0.891892

Fold2
As for Red: correct/predict = 40/58 = 0.689655
As for Blue: correct/predict = 11/23 = 0.478261
Short: accel/predictRedAB = 29/32 = 0.906250

10 < AverageVelocity <= 30:
Red/All = 103/161 = 0.639752
1 - Red/All = 58/161 = 0.360248
Red: Mean correct/predict = 0.628289
Blue: Mean correct/predict = 0.242857
Short: Mean accel/p